# Inverted Cart Pendulum Simulation

Only position of the cart is observed

In [1]:
import numpy as np
import control as ctrl
import matplotlib.pyplot as plt

# Continuous Time Model
State x = [position, velocity, angle, angular rate]

In [35]:
# Physical parameters 

m = 1 # pendulum mass
M = 5 # cart mass
L = 2 # length of pendulum
g = -9.81 
d = 1 # cart damping coefficient

# Simulation setup
t0 = 0
tf = 30
dt = 0.001                              # Sampling rate
T = np.arange(t0, tf+dt, dt)            # Simulation time array

# Initial conditions
x0 = np.array([-3, 0, np.pi+0.2,0])
# Desired state
xd = np.array([-1,0,np.pi,0])

x = np.zeros((4,len(T)))
x[:,0] = x0

u = np.zeros(len(T) -1)

# Desired state - adjusted for the linearization point
y = x0.copy()
yd = xd - np.array([0, 0, np.pi, 0])

# Error array - stores history of difference between measured desired state and predicted state
e = np.zeros((4, len(T)))

# Measurement array - storing history of position measurement
z = np.zeros(len(T))

# Kalman filter setup
mu0 = x0 - np.array([0, 0, np.pi, 0])   # Prior mean
S0 = 0.1 * np.eye(4)                    # A priori variance

mu = np.zeros((4, len(T)))              # Mean array - stores the history of state means
mu[:, 0] = mu0
S = S0

[-3.          0.          3.34159265  0.        ]


In [40]:
# Linearized system model at the top equilibrium point 
# (In other words, find the Jacobian of the motion model and evaluate at theta = pi and theta_dot = 0)
A = np.array([[0, 1, 0, 0],
              [0, -d/M, -m*g/M, 0],
              [0, 0, 0, 1],
              [0, -d/(M*L), -(m+M)*g/(M*L), 0]])

B = np.array([[0], 
              [1/M], 
              [0], 
              [1/(M*L)]])

C = np.array([[1, 0, 0, 0]])

D = np.array([[0]])

sys_ss = ctrl.ss(A, B, C, D)
sys_tf = ctrl.ss2tf(A, B, C, D)

lambda_ = np.linalg.eig(A)[0]
pc = np.linalg.matrix_rank(ctrl.ctrb(A, B))
print(f'Rank of controllability matrix: {pc:d}')
po = np.linalg.matrix_rank(ctrl.obsv(A, C))
print(f'Rank of observability matrix: {po:d}')

# Pole placement
# p = np.array([-0.01, -0.02, -0.03, -0.04])
# p = np.array([-0.1, -0.2, -0.3, -0.4])
p = np.array([-1, -1.1, -1.2, -1.3])
# p = np.array([-2, -2.1, -2.2, -2.3])
# p = np.array([-3, -3.1, -3.2, -3.3])
# p = np.array([-3.5, -3.6, -3.7, -3.8])


K = ctrl.place(A, B, p)

# Discretized model for EKF implementation
sysd = ctrl.c2d(sys_ss, dt, 'zoh')
Ad = sysd.A
Bd = sysd.B
Cd = sysd.C
Dd = sysd.D

# Noise
Q = 0.1 * np.eye(4)
R = 0.01

Rank of controllability matrix: 4
Rank of observability matrix: 4


In [37]:
for t in range(1, len(T)):
    # Error
    e[:, t] = yd - mu[:, t - 1]

    # Control input of the system
    u[t] = -np.dot(K, -e[:, t])

    # State update - Ground truth according to the system model
    q = np.sqrt(Q) * np.random.randn(4, 1)
    Sy = np.sin(y[2])       # Abbreviation for sin(theta)
    Cy = np.cos(y[2])       # Abbreviation for cos(theta)
    D_ = m * L * L * (M + m * (1 - Cy**2))
    
    dy = np.zeros(4)
    dy[0] = y[1]
    dy[1] = (1 / D_) * (-m**2 * L**2 * g * Cy * Sy + m * L**2 * (m * L * y[3]**2 * Sy - d * y[1])) + m * L * L * (1 / D_) * u[t]
    dy[2] = y[3]
    dy[3] = (1 / D_) * ((m + M) * m * g * L * Sy - m * L * Cy * (m * L * y[3]**2 * Sy - d * y[1])) - m * L * Cy * (1 / D_) * u[t]
    
    y += dy * dt #+q
    x[:, t] = y

    # Measurement model - State measurement according to the measurement model
    r = np.sqrt(R) * np.random.randn(1)
    z[t] = np.dot(C, (y - np.array([0, 0, np.pi, 0]))) + r

    # Kalman filter estimation
    # Prediction update
    mup = np.dot(Ad, mu[:, t - 1]) + np.dot(Bd, u[t])
    Sp = np.dot(Ad, np.dot(S, Ad.T)) + Q

    # Kalman gain
    Kg = np.dot(Sp, np.dot(Cd.T, np.linalg.inv(np.dot(Cd, np.dot(Sp, Cd.T)) + R)))

    # Measurement update
    mu[:, t] = mup + np.dot(Kg, (z[t] - np.dot(Cd, mup)))

    # Update state uncertainty
    S = np.dot(np.eye(4) - np.dot(Kg, Cd), Sp)


/tmp/ipykernel_43328/3821684634.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[t] = -np.dot(K, -e[:, t])
/tmp/ipykernel_43328/3821684634.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  z[t] = np.dot(C, (y - np.array([0, 0, np.pi, 0]))) + r


ValueError: could not broadcast input array from shape (4,4) into shape (4,)

In [ ]:
# Plotting Results
plt.figure()
plt.plot(T, z, label='$$z$$')
plt.plot(T, x[0, :], label='$$x$$', linewidth=2)
plt.plot(T, x[1, :], T, x[2, :], T, x[3, :])
plt.legend(loc='best', fontsize=10)
plt.xlabel('Time (seconds)')
plt.ylabel('State response')
plt.title('State response versus time (probabilistic and horizontal position feedback)')

plt.figure()
plt.subplot(2, 1, 1)
plt.plot(T, mup[0, :])
plt.legend([r'$$\overline{\mu_{1}}$$'], loc='best', fontsize=12)
plt.xlabel('Time (seconds)')
plt.ylabel('Prediction of horizontal position')
plt.title('Prediction of horizontal position of the cart vs time')

plt.subplot(2, 1, 2)
plt.plot(T, mu[0, :], 'r')
plt.legend([r'$$\mu_{1}$$'], loc='best', fontsize=12)
plt.xlabel('Time (seconds)')
plt.ylabel('Belief of horizontal position')
plt.title('Belief of horizontal position of the cart vs time')

plt.figure()
plt.plot(T[:-1], u)
plt.legend(['u'], loc='best')
plt.xlabel('Time (seconds)')
plt.ylabel('Control input')
plt.title('Control